### Training data preparation for Yolov5 bounding box detection

Pre-requisites:

- ensure you have download the DICOM data (download_data_lidc.ipynb)

In [ ]:
!pip install -r requirements.txt

In [12]:
import pylidc as pl
from pylidc.utils import consensus
import pydicom as dicom
from skimage.measure import find_contours
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import contextlib
import matplotlib.patches as patches
import os
import imageio
import sys
from glob import glob
import os
import cv2
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
import os
from numpy import random
import time
import pandas as pd
from PIL import Image
import contextlib
import os
import contextlib
import imageio
from __future__ import print_function
import sys
from contextlib import redirect_stdout, redirect_stderr

#### Routines for bounding box extraction

Define some routines to get the bounding around for the masks for the image

In [13]:
import matplotlib.patches as patches
def get_bounding_box(img):
    minx = []
    miny = []
    maxx = []
    maxy = []
    contours = find_contours(img.astype(float), 0.5)
    for contour in contours:
        minx.append(np.min(contour[:,1]))
        maxx.append(np.max(contour[:,1]))
        miny.append(np.min(contour[:,0]))
        maxy.append(np.max(contour[:,0]))
        
    if len(minx) > 0:
        return [(a, b, w-a, h-b) for a, b, w, h in zip(minx, miny, maxx, maxy)]
    else:
        return None
    
def plot_bounding_box(img):
    fig, ax = plt.subplots()
    ax.imshow(img, cmap=plt.cm.gray)
    
    boxes = get_bounding_box(img)
    if boxes:
        for box in boxes:
            x,y,w,h = box
            print("x = {}, y = {}, w = {}, h= {}".format(x,y,w,h))
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

Routine to find all bounding boxes for the mask and plotting

In [14]:
def find_all_bboxes(nodule_annotation, nodule_id, plot=False):
    bboxes = []
    #cmask, cbbox, masks = consensus(nodule_annotation)
    
    cmask_CT = []
    cbbox_CT = []
    masks_CT = []
    for nodule_idx, nodule in enumerate(nodules_annotation):
        cmask, cbbox, masks = consensus(nodule)
        cmask_CT.append(cmask)
        cbbox_CT.append(cbbox)
        masks_CT.append(masks)
    CT_mask = np.zeros_like(vol)    
    nodule_num = len(cmask_CT)
    print("Number of nodules for patient {} is {}".format(patient_id, nodule_num))
    
    for i in range(nodule_num):
        cmask = cmask_CT[i]
        cbbox = cbbox_CT[i]
        masks = masks_CT[i]
        CT_mask[cbbox] += cmask
        

    print("There are {} slices in the image".format(cbbox_CT[nodule_id][2].stop - cbbox_CT[nodule_id][2].start))
    for idx in np.arange(cbbox_CT[nodule_id][2].start, cbbox_CT[nodule_id][2].stop):
        boxes = get_bounding_box(CT_mask[:,:,idx])
        if plot:
            plot_bounding_box(CT_mask[:,:,idx])
        bboxes.append(boxes)
    return bboxes

Routine to plot the bounding box around the mask in the full size mask

In [15]:
def plot_bbox_on_full_size_mask(vol, nodules_annotation, nodule_id):
    cmask_CT = []
    cbbox_CT = []
    masks_CT = []
    for nodule_idx, nodule in enumerate(nodules_annotation):
        cmask, cbbox, masks = consensus(nodule)
        cmask_CT.append(cmask)
        cbbox_CT.append(cbbox)
        masks_CT.append(masks)
    CT_mask = np.zeros_like(vol)    
    nodule_num = len(cmask_CT)
    print("Number of nodules for patient is {}".format(nodule_num))
    
    for i in range(nodule_num):
        cmask = cmask_CT[i]
        cbbox = cbbox_CT[i]
        masks = masks_CT[i]
        CT_mask[cbbox] += cmask
        
    for slice_idx in np.arange(cbbox_CT[nodule_id][2].start, cbbox_CT[nodule_id][2].stop):
        plot_bounding_box(CT_mask[:,:,slice_idx])

Routine to plot the bounding box around the nodule using the mask bounding box coordinates

In [16]:
def plot_bbox_on_image(vol, nodules_annotation, nodule_id):
    cmask_CT = []
    cbbox_CT = []
    masks_CT = []
    for nodule_idx, nodule in enumerate(nodules_annotation):
        cmask, cbbox, masks = consensus(nodule)
        cmask_CT.append(cmask)
        cbbox_CT.append(cbbox)
        masks_CT.append(masks)
    CT_mask = np.zeros_like(vol)    
    nodule_num = len(cmask_CT)
    print("Number of nodules for patient is {}".format(nodule_num))
    
    for i in range(nodule_num):
        cmask = cmask_CT[i]
        cbbox = cbbox_CT[i]
        masks = masks_CT[i]
        CT_mask[cbbox] += cmask
        
    for slice_idx in np.arange(cbbox_CT[nodule_id][2].start, cbbox_CT[nodule_id][2].stop):
        boxes = get_bounding_box(CT_mask[:,:,slice_idx])
        img = vol[:,:,slice_idx]
        fig, ax = plt.subplots()
        ax.imshow(img, cmap=plt.cm.gray)
        if boxes:
            for box in boxes:
                x, y, w, h = box
                print("x = {}, y = {}, w = {}, h= {}".format(x,y,w,h))
                rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
                ax.add_patch(rect)

### Prepare the dataset for YoloV5 training

Routine to create the image and the labels (based on bounding boxes) for Yolov5 training

#### Clear the contents of the generated images and labels for Yolov5

In [17]:
import shutil
def clear_dirs(paths):
    for path in paths:
        if os.path.isdir(path):
            shutil.rmtree(path)

##### For all the patients we downloaded, extract the bounding box and create the image and labels for YoloV5 training

### Save image label and mask

- For each CT image, save the image, the mask and the bounding box label
- For bounding box dataset, save empty files for images with no mask.

Routine to suppress stdout and stderr messages

In [18]:
class RedirectStdStreams(object):
    def __init__(self, stdout=None, stderr=None):
        self._stdout = stdout or sys.stdout
        self._stderr = stderr or sys.stderr

    def __enter__(self):
        self.old_stdout, self.old_stderr = sys.stdout, sys.stderr
        self.old_stdout.flush(); self.old_stderr.flush()
        sys.stdout, sys.stderr = self._stdout, self._stderr

    def __exit__(self, exc_type, exc_value, traceback):
        self._stdout.flush(); self._stderr.flush()
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr

In [19]:
def un(a, b):
    return np.union1d(a, b)

def unr(ll, acc, i):
    if i < len(ll):
        j = i + 1
        acc = un(ll[i], acc)
        return unr(ll, acc, j)
    else:
        return acc

def get_slices_with_masks(cbbox_CT):
    ll = []
    for cb in cbbox_CT:
        ll.append(np.arange(cb[2].start, cb[2].stop+1))
    return unr(ll, [], 0)

#### Preprocessing routine for images

In [20]:
#code ref: https://github.com/JenifferWuUCLA/pulmonary-nodules-segmentation/blob/master/tianchi_segment_lung_ROI.ipynb
def preprocess_image(img):
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    # Find the average pixel value near the lungs
    #　to renormalize washed out images
    #middle = img[100:400, 100:400]
    middle = img
    mean = np.mean(middle)
    max = np.max(img)
    min = np.min(img)
    # To improve threshold finding, I'm moving the
    #　underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    # Using Kmeans to separate foreground (radio-opaque tissue)
    #　and background (radio transparent tissue ie lungs)
    # Doing this only on the center of the image to avoid
    #　the non-tissue parts of the image as much as possible
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle, [np.prod(middle.shape), 1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold, 1.0, 0.0)  # threshold the image
    eroded = morphology.erosion(thresh_img, np.ones([4, 4]))
    dilation = morphology.dilation(eroded, np.ones([10, 10]))
    labels = measure.label(dilation)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0] < 475 and B[3]-B[1] < 475 and B[0] > 40 and B[2] < 472:
            good_labels.append(prop.label)

    mask = np.zeros_like(labels)
    for N in good_labels:
        mask = mask + np.where(labels == N, 1, 0)

    mask = morphology.dilation(mask, np.ones([10, 10]))  # one last dilation
    return img * mask

##### Preprocessing steps:

We will save  512x512 images and the corresponding labels in Yolov5 pytorch format 

- Mark the mask for nodules which were annotated by all 4 radiologists
- Select all the slices which have annotated masks
- Select an equal number of slices which have no annotated masks (for negative images)
- Create the bounding box label for each slice with a mask using find coutours method
- Process the image to highlight the region of interest
- Save the processed image and the labels

In [21]:
"""
Save images and labels for nodules that have annotations from all 4 radiologists
"""
def save_image_mask_label(patient_id, imagedirname, labeldirname, maskdirname, temp_dir, normalize=False):
    
    # For supressing stdout and stderr messages
    devnull = open(os.devnull, 'w')
    
    # Read the volume (3D-dicom image) for the patient
    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id.in_([patient_id]))[0]
    nodules_annotation = scan.cluster_annotations()
    try:
        with contextlib.redirect_stdout(None):
            vol = scan.to_volume()
    except:
        print("Error loading DICOM for patient {}".format(patient_id))
        return
    
    # Process the mask for the image
    cmask_CT = []
    cbbox_CT = []
    masks_CT = []
    for nodule_idx, nodule in enumerate(nodules_annotation):
        if len(nodule) >=4:
            cmask, cbbox, masks = consensus(nodule)
            cmask_CT.append(cmask)
            cbbox_CT.append(cbbox)
            masks_CT.append(masks)
        
    if len(cbbox_CT) <=0:
        print("No relevant nodules for patient {}".format(patient_id))
        return
    
    CT_mask = np.zeros_like(vol)    
    nodule_num = len(cmask_CT)
      
    for i in range(nodule_num):
        cmask = cmask_CT[i]
        cbbox = cbbox_CT[i]
        masks = masks_CT[i]
        CT_mask[cbbox] += cmask
    
    # Get the slices that have annotations
    slices_with_mask = get_slices_with_masks(cbbox_CT)

    idxs = np.arange(CT_mask.shape[2])
    
    # Get all slices that have no annotations
    compl_idxs = [idx for idx in idxs if idx not in slices_with_mask]
    SEED = 2021
    np.random.seed(SEED)
    # Random sample the slices with no annotations
    np.random.shuffle(compl_idxs)
    # Select equal number of non annotated slices as the number of annotated slices
    compl_idxs = compl_idxs[:len(slices_with_mask)]
    all_slices = np.union1d(slices_with_mask, compl_idxs)
    # Save the images and masks and labels for only slices_with_mask and compl_idxs
    all_slices = all_slices.astype(int)

    # Iterate through all slices and save the images
    z_slice_cnt = 0
    for z_slice in all_slices:
        im = vol[:,:,z_slice]
        # mean normalize the image to prevent unit8 conversion errors when saving with imageio
        if normalize:
            im = im - np.mean(im)
            im = im / np.std(im)
        try:
            # in order to preserve the richness of the image, imageio is used to save and then read the image back from the disk
            # TODO: could potentially do it in memory as well
            # TODO: create the temp folder
            imageio.imwrite(temp_dir + '/temp_img.jpg', im)
            disk_img = cv2.imread(temp_dir + '/temp_img.jpg')
        except:
            continue
        
        im = cv2.cvtColor(disk_img, cv2.COLOR_BGR2GRAY)
        img_t = preprocess_image(im)
        
        image_filename = "{}_{}.jpg".format(patient_id, z_slice)
        with RedirectStdStreams(stdout=devnull, stderr=devnull):
            imageio.imwrite(imagedirname + "/" + image_filename, img_t)
        z_slice_cnt += 1
    
    # Save bounding box labels
    for k_slice in all_slices:
        msk = CT_mask[:,:,k_slice]
        boxes = get_bounding_box(msk)
        if boxes:
            for box in boxes:
                x, y, w, h = box
                # Drop small objects
                if (w <= 3) or (h <= 3):
                    continue
                center_x = (x + ((w)/2))/vol[:,:,k_slice].shape[0]
                center_y = (y + ((h)/2))/vol[:,:,k_slice].shape[1]
                width = w/vol[:,:,k_slice].shape[0]
                height = h/vol[:,:,k_slice].shape[1]

                c = 0 # there is only one class, the 'nodule'
                # Save this as the label for the image for the YoloV5 object detection training
                label = "{} {} {} {} {}".format(c, center_x, center_y, width, height)
                label_filename = "{}_{}.txt".format(patient_id, k_slice)

                f = open(labeldirname + "/" + label_filename, "a")
                f.write(label + "\n")
                f.close()
        else:
            # Save this as the label for the image for the YoloV5 object detection training
            label_filename = "{}_{}.txt".format(patient_id, k_slice)
            f = open(labeldirname + "/" + label_filename, "a")
            f.write("")
            f.close()

In [22]:
def create_dataset(patient_list, dicom_dir, root_data_dir, dtype='train'):
    # Set the pylidc data config
    print("Setting DICOM dir = {}".format(dicom_dir))
    f = open ('/root/.pylidcrc','w')    #For Colab
    f.write('[dicom]'+'\n')
    f.write('path =' + dicom_dir +'\n')
    f.write('warn = True')
    f.close()
    
    image_path = root_data_dir + '/' + dtype + '/images'
    mask_path = root_data_dir + '/' + dtype + '/masks'
    label_path = root_data_dir + '/' + dtype + '/labels'
    
    print("Saving images to {}, masks to {} and labels to {}".format(image_path, mask_path, label_path))
    #create temp dir
    temp_dir = '/app/data/temp/'
    if not os.path.exists(temp_dir):
        print("Removing and then creating directory {}".format(temp_dir))
        os.makedirs(temp_dir)
    else:
        print("Directory {} present!".format(temp_dir))
    for idx, patient in tqdm(enumerate(patient_list)):
        if not os.path.exists(image_path):
            print("Removing and then creating directory {}".format(image_path))
            os.makedirs(image_path)
        if not os.path.exists(label_path):
            print("Removing and then creating directory {}".format(label_path))
            os.makedirs(label_path)
        if not os.path.exists(mask_path):
            print("Removing and then creating directory {}".format(mask_path))
            os.makedirs(mask_path)
        save_image_mask_label(patient, image_path, label_path, mask_path, temp_dir, normalize=True)

In [23]:
patient_list_train = os.listdir('/app/data/lidc/dicom/train/sorted_gcsfiles/')
print("Train patients = {}".format(len(patient_list_train)))
patient_list_test = os.listdir('/app/data/lidc/dicom/test/sorted_gcsfiles/')
print("Test patients = {}".format(len(patient_list_test)))
patient_list_val = os.listdir('/app/data/lidc/dicom/val/sorted_gcsfiles/')
print("Val patients = {}".format(len(patient_list_val)))

Train patients = 51
Test patients = 48
Val patients = 51


In [24]:
%%time
create_dataset(patient_list_train, '/app/data/lidc/dicom/train/sorted_gcsfiles/', '/app/data/yolov5/processed/', 'train')

0it [00:00, ?it/s]

Setting DICOM dir = /app/data/lidc/dicom/train/sorted_gcsfiles/
Saving images to /app/data/yolov5/processed//train/images, masks to /app/data/yolov5/processed//train/masks and labels to /app/data/yolov5/processed//train/labels
Directory /app/data/temp/ present!


Lossy conversion from float64 to uint8. Range [-1.6175465976867516, 4.7134925666933984]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6172036642270204, 4.7185491813223495]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6153021579639926, 2.523790519548622]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6176168803245081, 2.4368753261217386]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6158951763772116, 2.4498136008637594]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.574636746337801, 3.7298482598818756]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.5767018530964565, 3.74483421

No relevant nodules for patient LIDC-IDRI-0977


3it [00:12,  5.66s/it]

...

No relevant nodules for patient LIDC-IDRI-0506


50it [05:14,  5.90s/it]

No relevant nodules for patient LIDC-IDRI-0136


Lossy conversion from float64 to uint8. Range [-1.820006294965846, 1.8994784565438445]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.820400474369072, 2.1923548696572164]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8146012485201857, 2.1129737275074687]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8119504958276478, 2.1785427228312955]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7880497811521323, 2.3297093769127293]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7887314715701417, 2.334154799485527]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7888203623295078, 2.396226737

CPU times: user 8min 47s, sys: 33.2 s, total: 9min 20s
Wall time: 5min 27s


In [25]:
%%time
create_dataset(patient_list_val, '/app/data/lidc/dicom/val/sorted_gcsfiles/', '/app/data/yolov5/processed', 'val')

0it [00:00, ?it/s]

Setting DICOM dir = /app/data/lidc/dicom/val/sorted_gcsfiles/
Saving images to /app/data/yolov5/processed/val/images, masks to /app/data/yolov5/processed/val/masks and labels to /app/data/yolov5/processed/val/labels
Directory /app/data/temp/ present!


1it [00:01,  1.13s/it]

No relevant nodules for patient LIDC-IDRI-0534


Lossy conversion from float64 to uint8. Range [-1.6930709569166136, 2.4964654686989753]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7070585043251807, 2.3546711016160358]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7014093591684363, 2.4208774875107584]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6439881881720981, 2.6274801855642917]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6468361680798156, 2.607554546678696]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6475848632595749, 2.5741718088888295]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6492268887515509, 2.5450793

...

No relevant nodules for patient LIDC-IDRI-0483


Lossy conversion from float64 to uint8. Range [-1.8497727594634807, 1.780063441458374]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.85046149636423, 1.747093445851866]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.848905112696023, 1.8511973828518418]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8462066477002401, 1.8148763224264315]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8443251198511825, 1.7343922746566447]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8441034392398215, 1.6871742103628902]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8439760458750685, 1.70759901501

No relevant nodules for patient LIDC-IDRI-0110


Lossy conversion from float64 to uint8. Range [-1.6968246108731146, 1.8823995399100328]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7033458610345071, 1.8157012290497732]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7124300861000454, 2.07976022798378]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7128000810150406, 1.9546412889803462]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6912345487990028, 1.9301591402006821]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.655746996783085, 2.1644337773382327]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6252550498616318, 2.444025189

CPU times: user 9min 16s, sys: 34.2 s, total: 9min 50s
Wall time: 5min 45s


In [26]:
%%time
create_dataset(patient_list_test, '/app/data/lidc/dicom/test/sorted_gcsfiles/', '/app/data/yolov5/processed', 'test')

0it [00:00, ?it/s]

Setting DICOM dir = /app/data/lidc/dicom/test/sorted_gcsfiles/
Saving images to /app/data/yolov5/processed/test/images, masks to /app/data/yolov5/processed/test/masks and labels to /app/data/yolov5/processed/test/labels
Directory /app/data/temp/ present!


1it [00:01,  1.03s/it]

No relevant nodules for patient LIDC-IDRI-0872


Lossy conversion from float64 to uint8. Range [-1.6326088419583928, 4.740325552976174]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.635273886127258, 4.730570197026122]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6375342587463717, 4.673219687827032]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6419003420689575, 4.630210632803526]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6492709022496834, 4.61358316536731]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6496953707818005, 4.61056914313486]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.6491655699438612, 4.608967511775813

...

No relevant nodules for patient LIDC-IDRI-0864


Lossy conversion from float64 to uint8. Range [-1.78561571490769, 1.9663479645506887]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7814064446479139, 1.9891209338043634]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.782401573140465, 2.0025223414095525]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.78174717939479, 2.030793925980064]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7896290403121344, 2.4597648786155473]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7897817044155464, 2.4930275775166058]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7841667466496873, 2.459981609992

No relevant nodules for patient LIDC-IDRI-0125


Lossy conversion from float64 to uint8. Range [-1.8494995129947616, 3.1350377689979494]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.8563437894590649, 1.7368094531093419]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7864111006171792, 2.0104700601523335]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.787820518472662, 2.158059274567829]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.783490036670571, 2.2635464982628455]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7854918287023436, 2.194897718789922]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.7865255673191853, 2.1779761842

No relevant nodules for patient LIDC-IDRI-0028
CPU times: user 12min 28s, sys: 37.2 s, total: 13min 5s
Wall time: 7min 23s
